# Lehmer 指標とその計算

マチンは $\arctan$ を使った円周率公式

$$\frac{\pi}{4} = 4 \arctan \frac{1}{5} -\arctan \frac{1}{239}$$

を発見し、円周率を100桁目まで計算しました。この公式はそれ以前に発見されていた公式よりも計算効率が遥かによく、この公式を改良した公式がたくさん発見されました。一般に

$$\begin{gather*}
k\frac{\pi}{4} = \sum_{i=1}^{N} c_i \arctan \frac{a_i}{b_i} \\(k \in \mathbb{Z}_{>0},\ c_i \in \mathbb{Z} \setminus \{0\},\ a_i, b_i \in \mathbb{Z},\ a_i < b_i)
\end{gather*}$$

という形の公式を $\arctan$ 公式、またはマチン系の公式といいます。

Lehmer は $\arctan$ 公式の計算効率の良さの指標を

$$\sum_{i=1}^N\frac{1}{\log_{10} \frac{b_i}{a_i}}$$

と定め、Lehmer 指標を減少させる方法を考案しました。これをいくつかの公式で確かめます。

In [2]:
import sys
from typing import List
sys.path.append('/home/jovyan/work/')
from lib.arctan import ArctanParam
from pydantic import BaseModel, Field
import math


class ArctanFormulaParam(BaseModel):
    k: int = Field(..., ge=1)
    params: List[ArctanParam]
        
    def is_valid(self) -> bool:
        """
        arctan 公式の両辺が等しいことを確認する
        (本当は複素数を使った方が楽)
        """
        # k > 1 だとエラー
        left = ArctanParam(coeff = self.k, numer = 1, denom = 1).composit()
        
        right = self.params[0]
        for p in self.params[1:]:
            right = right.add(p)
        
        if left.equal_to(right):
            return True
        else:
            return False
        
        
def lehmer_measure(at_form_param: ArctanFormulaParam) -> float:
    return sum([
        1 / math.log10(1 / p.x)
        for p in at_form_param.params
    ])
        
    
def formula_str(at_form_param: ArctanFormulaParam) -> str:
    def coeff_str(coeff: int) -> str:
        s = ""
        if coeff > 0:
            s = "+ "
        else:
            s = "- "
        if abs(coeff) > 1:
            s = s + str(abs(coeff))
        return s
    
    def arctan_str(p: ArctanParam) -> str:
        return f" {coeff_str(p.coeff)} arctan {p.numer}/{p.denom}"
        
    arctans_str = "".join([
        arctan_str(p)
        for p in at_form_param.params
    ])
    return f"{coeff_str(at_form_param.k)[1:]}π/4 = {arctans_str[3:]}"
    

In [3]:
machin = ArctanFormulaParam(
    k = 1, 
    params = [
        ArctanParam(coeff=4, numer=1, denom=5), 
        ArctanParam(coeff=-1, numer=1, denom=239)
    ]
)

euler = ArctanFormulaParam(
    k = 1,
    params = [
        ArctanParam(coeff=5, numer=1, denom=7), 
        ArctanParam(coeff=2, numer=3, denom=79)
    ]
)

gauss = ArctanFormulaParam(
    k = 1,
    params = [
        ArctanParam(coeff=12, numer=1, denom=18), 
        ArctanParam(coeff=8, numer=1, denom=57),
        ArctanParam(coeff=-5, numer=1, denom=239),
    ]
)

klingenstierna = ArctanFormulaParam(
    k = 1,
    params = [
        ArctanParam(coeff=8, numer=1, denom=10), 
        ArctanParam(coeff=-1, numer=1, denom=239),
        ArctanParam(coeff=-4, numer=1, denom=515),
    ]
)

takano = ArctanFormulaParam(
    k = 1,
    params = [
        ArctanParam(coeff=12, numer=1, denom=49), 
        ArctanParam(coeff=32, numer=1, denom=57),
        ArctanParam(coeff=-5, numer=1, denom=239),
        ArctanParam(coeff=12, numer=1, denom=110443)
    ]
)

stormer = ArctanFormulaParam(
    k = 1,
    params = [
        ArctanParam(coeff=44, numer=1, denom=57), 
        ArctanParam(coeff=7, numer=1, denom=239),
        ArctanParam(coeff=-12, numer=1, denom=682),
        ArctanParam(coeff=24, numer=1, denom=12943)
    ]
)

param_dikt = {
    "machin": machin,
    "euler": euler,
    "klingen-": klingenstierna,
    "gauss": gauss,
    "takano": takano,
    "störmer": stormer
}

for k, v in param_dikt.items():
    s = f"{k}:\t {lehmer_measure(v)} : \t {formula_str(v)}"
    if v.is_valid():
        s = s + "\t ○"
    print(s)

machin:	 1.8511276523168558 : 	  π/4 = 4 arctan 1/5 -  arctan 1/239	 ○
euler:	 1.8872692426749564 : 	  π/4 = 5 arctan 1/7 + 2 arctan 3/79	 ○
klingen-:	 1.7892088692397219 : 	  π/4 = 8 arctan 1/10 -  arctan 1/239 - 4 arctan 1/515	 ○
gauss:	 1.7866075340193157 : 	  π/4 = 12 arctan 1/18 + 8 arctan 1/57 - 5 arctan 1/239	 ○
takano:	 1.7799043271373138 : 	  π/4 = 12 arctan 1/49 + 32 arctan 1/57 - 5 arctan 1/239 + 12 arctan 1/110443	 ○
störmer:	 1.5860413585818451 : 	  π/4 = 44 arctan 1/57 + 7 arctan 1/239 - 12 arctan 1/682 + 24 arctan 1/12943	 ○


## Lehmer 指標を減らす方法

Lehmer は Lehmer 指標を減らす方法として、自然数 $n$ に対して

$$
\arctan \frac{a}{b} = n \arctan \frac{a}{nb} -\arctan u_n \left(\frac{b}{a}\right)
$$

という変形を考え、$1 \leq \frac{b}{a} \leq n^2$ のときに Lehmer 指標が減ることを示しました。$\frac{a}{b} = \frac{1}{x}$ とおいて

$$
\arctan \frac{1}{x} = n \arctan \frac{1}{nx} -\arctan u_n (x)
$$

とした方がわかりやすいので、以下はこの形で考えます。$u_n(x)$ は一般に

$$
u_n(x) = \frac{\displaystyle
-\sum_{\substack{0 \leq k \leq n, \\ k: 偶数}}(-1)^{\frac{k}{2}} \frac{k(n+1)}{k+1} \binom{n}{k} n^{n-k-1}x^{n-k}
}{
\displaystyle n^n x^{n+1} + \sum_{\substack{0 \leq k \leq n, \\ k: 奇数}} (-1)^{\frac{k-1}{2}} \frac{k(n+1)}{k+1} \binom{n}{k}n^{n-k-1}x^{n-k}
}
$$

と表されます。

In [32]:
from fractions import Fraction
from scipy.special import comb


class LehmerRationalExpress():
    def __init__(self, n: int):
        self.n = n
        
    def _numer_coeff(self) -> List[int]:
        coeffs = []
        for k in range(2, self.n + 1, 2):
            coeffs.append(
                (-1) * (-1) ** (k / 2) * (k * (self.n + 1) / (k +1))
                * comb(self.n, k) * self.n ** (self.n - k -1)
            )
        return coeffs
        
    def _denom_coeff(self) -> List[int]:
        coeffs = [self.n ** self.n]
        
        for k in range(1, self.n + 1, 2):
            coeffs.append(
                (-1) ** ((k -1) / 2) * (k * (self.n + 1) / (k +1))
                * comb(self.n, k) * self.n ** (self.n - k -1)
            )
        return coeffs
        
    def __str__(self):
        numer = " + ".join([
            f"{c}x^{self.n - 2 * (i+1)}"
            for i, c in enumerate(self._numer_coeff())
        ])
        denom = " + ".join([
            f"{c}x^{self.n - 2 * i + 1}"
            for i, c in enumerate(self._denom_coeff())
        ])
        return f"{numer} / {denom}"
        
    def value(self, numer: int, denom: int) -> Fraction:
        # 逆にする
        x = Fraction(denom, numer)
        
        numer = Fraction(0, 1)
        for i, c in enumerate(self._numer_coeff()):
            numer = numer + int(c) * x ** (self.n - 2 * (i+1))
            
        denom = Fraction(0, 1)
        for i, c in enumerate(self._denom_coeff()):
            denom = denom + int(c) * x ** (self.n - 2 * i + 1)

        return numer / denom

In [7]:
u_2 = LehmerRationalExpress(2)
u_3 = LehmerRationalExpress(3)
u_4 = LehmerRationalExpress(4)
print(f"u_2 = {u_2}")
print(f"u_3 = {u_3}")
print(f"u_4 = {u_4}")

u_2 = 1.0x^0 / 4x^3 + 3.0x^1
u_3 = 8.0x^1 / 27x^4 + 18.0x^2 + -1.0x^0
u_4 = 80.0x^2 + -1.0x^0 / 256x^5 + 160.0x^3 + -15.0x^1


オイラーの公式の $\frac{1}{7}$ の項を変形して Lehmer 指標が小さくなるか確かめてみます。$3^2 = 9$ なので、$n \geq 3$ ならば減ります。

In [45]:
u_2 = LehmerRationalExpress(2)
u_2x = u_2.value(1, 7)
euler2 = ArctanFormulaParam(
    k = 1,
    params = [
        ArctanParam(coeff=2, numer=3, denom=79),
        ArctanParam(coeff=5 * 2, numer=1, denom=7 * 2), 
        ArctanParam(coeff=-5, numer=u_2x.numerator, denom=u_2x.denominator), 
    ]
)


u_3 = LehmerRationalExpress(3)
u_3x = u_3.value(1, 7)
euler3 = ArctanFormulaParam(
    k = 1,
    params = [
        ArctanParam(coeff=2, numer=3, denom=79),
        ArctanParam(coeff=5 * 3, numer=1, denom=7 * 3), 
        ArctanParam(coeff=-5, numer=u_3x.numerator, denom=u_3x.denominator), 
    ]
)

u_5 = LehmerRationalExpress(5)
u_5x = u_5.value(1, 7)
euler5 = ArctanFormulaParam(
    k = 1,
    params = [
        ArctanParam(coeff=2, numer=3, denom=79),
        ArctanParam(coeff=5 * 5, numer=1, denom=7 * 5), 
        ArctanParam(coeff=-5, numer=u_5x.numerator, denom=u_5x.denominator), 
    ]
)

param_dikt = {
    "euler": euler,
    "euler2": euler2,
    "euler3": euler3,
    "euler5": euler5
}

for k, v in param_dikt.items():
    s = f"{k}:\t {lehmer_measure(v)} : \t {formula_str(v)}"
    if v.is_valid():
        s = s + "\t ○"
    print(s)

euler:	 1.8872692426749564 : 	  π/4 = 5 arctan 1/7 + 2 arctan 3/79	 ○
euler2:	 1.89454855296668 : 	  π/4 = 2 arctan 3/79 + 10 arctan 1/14 - 5 arctan 1/1393	 ○
euler3:	 1.786072140579028 : 	  π/4 = 2 arctan 3/79 + 15 arctan 1/21 - 5 arctan 14/16427	 ○
euler5:	 1.681035382331817 : 	  π/4 = 2 arctan 3/79 + 25 arctan 1/35 - 5 arctan 42854/46517997	 ○


ちなみに、$u_n(x)$ の分母の最大次の項の係数が $n^n$ なので、$n$ に関して爆発的に大きくなります。よって $n$ を大きくするのは実用的に良くないかもしれません。例えば $n = 10$ とすると、

$$\frac{\pi}{4} = 2 \arctan \frac{3}{79} + 50 \arctan \frac{1}{70} - 5 \arctan \frac{19002106518564401}{19994397349132084193}$$

となります。$n = 100$ なら $100^100$ 桁を超えます。 

In [59]:
N = 10
u_10 = LehmerRationalExpress(N)
u_10x = u_10.value(1, 7)
euler5 = ArctanFormulaParam(
    k = 1,
    params = [
        ArctanParam(coeff=2, numer=3, denom=79),
        ArctanParam(coeff=5 * N, numer=1, denom=7 * N), 
        ArctanParam(coeff=-5, numer=u_10x.numerator, denom=u_10x.denominator), 
    ]
)

print(f"{lehmer_measure(euler5)} : \t {formula_str(euler5)} : valid {euler5.is_valid()}")

1.5768462159565821 : 	  π/4 = 2 arctan 3/79 + 50 arctan 1/70 - 5 arctan 19002106518564401/19994397349132084193 : valid True
